## StreamStats API Scraper Automatic

__Description__: Tool to automatically run the [USGS StreamStats tool](https://www.usgs.gov/mission-areas/water-resources/science/streamstats-streamflow-statistics-and-spatial-analysis-tools?qt-science_center_objects=0#qt-science_center_objects) for multiple points within a catchment and return the flow frequency curves and subcatchment boundaries.

__Input__: A shapefile containing the latitude and longitude of points on the stream grid for the specified state (confluence and main stem locations).

__Output__: GeoJSON file containing the delinated catchment boundary and flow frequency data for each point, as well as a CSV file containing the flow frequency data.

*Authors*: sputnam@Dewberry.com & slawler@Dewberry.com

### Load libraries and Python options:

In [34]:
import os
import re
import sys
import json
sys.path.append('../USGStools')
from StreamStats_API_Scraper import*
import geopandas as gpd
from geojson import dump

### Specify the state abbreviation and location of the shapefile: 

##### Specify:

In [2]:
state='NY' #The state abbreviation in uppercase

path=r'C:\Users\sputnam\Documents\GitHub\usgs-tools\StreamStats\results\04150303' #Specify the location of the shapefile containing the lat/lon of points on the stream grid

name='04150303_Confluences_Scoped.shp' #The name of the shapefile

use_epsg='4326' #Specify a consistent coordinate reference system

allresults=os.path.join(path,'AllStreamStats') #Location to save the StreamStats results for each polygon

if os.path.isdir(allresults)==False: #If the desired path does not exist, create it.
    os.mkdir(allresults)    

##### Load the shapefile:

In [3]:
gdf=gpd.read_file(os.path.join(path, name)) #Read the shapefile as a geopandas dataframe

gdf=gdf.to_crs({'init': 'epsg:{0}'.format(use_epsg)}) #Transform the coordinate reference system of the geodataframe

### Run the API tool for each point:

In [6]:
polyg={} #Dictionary to store the catchment polygons (catchment boundaries) 

ffdata={} #Dictionary to store the outlet flow frequency data dictionaries

get_flow=True
print_status=True

if state=='WI': get_flow=False 

start_confluence=0 #The confluence number to start. Normally set to zero unless there was an issue   
stop_confluence=100

for i in gdf[gdf[gdf['num']==start_confluence].index[0]:gdf[gdf['num']==stop_confluence].index[0]+1].index:
    lon, lat, num = gdf.geometry[i].x, gdf.geometry[i].y, gdf['num'][i] #Longitude and latitude for each shapely point and the confluence number
    if print_status: print("Lat/Lon/Confluence:", lat, lon, num)
    polyg[num], ff_json  = SS_scrape(state, lon, lat, use_epsg, print_status) #Run the SS_scrape function. Option: set status=False to hide print statements
    if get_flow: 
        ffdata[num]= get_peaks(ff_json) #Use the function above to extract the json data
        polyg[num]['features'][0]['ffcurve']=ffdata[num]
    with open(os.path.join(allresults,'StreamStats_Polygons_{0}.geojson'.format(num)), 'w') as f:
       dump(polyg[num], f)   

### Load the results:

In [37]:
poly_files=[] #Empty list to store the geojson paths
ffdata={} 

poly_files=load_all_results(allresults)

gdf=gpd.GeoDataFrame(crs={'init': 'epsg:{}'.format(use_epsg)})
                          
for _,filename in enumerate(poly_files):
    num=re.findall('\d+', filename)
    temp_df=gpd.read_file(filename)
    temp_df['ID_Num']=int(num[-1])
    gdf=gdf.append(temp_df.iloc[0])
    with open(filename) as f:
        data = json.load(f)
        ffdata[num[-1]]=data['features'][0]['ffcurve']

559 Polygon Files Found


###  Save:

##### The flow frequency data as a CSV:

In [39]:
if get_flow: ffdata_df=ff_summary(ffdata) #Run this function to construct the summary table for all outlet locations
    
if get_flow: ffdata_df.to_csv(os.path.join(path,'StreamStats_FlowFrequency.csv')) #Save the results as a csv

                0          1         10        100        101       102  \
RI                                                                        
1.25  3269.042312  25.088418   7.003959   4.238119   4.167097  2.642965   
1.5   3641.973430  30.849801   8.494574   5.113636   5.091586  3.243820   
2.0   4080.529298  38.621843  10.485974   6.275664   6.358985  4.079904   
5.0   5240.037297  60.316559  16.007592   9.530964   9.865393  6.406996   
10.0  6025.059073  76.213688  20.017580  11.951031  12.410024  8.121865   

            103        104         105        106    ...             90  \
RI                                                   ...                  
1.25   6.959253   3.483064   46.896389   3.788126    ...      29.955113   
1.5    8.489727   4.251320   56.757388   4.653818    ...      37.137951   
2.0   10.540833   5.294715   69.570617   5.843961    ...      46.970890   
5.0   16.306161   8.226494  105.276502   9.226237    ...      74.932082   
10.0  20.595382  10.4176

##### The catchment polygons as a Shapefile:

In [8]:
gdf.to_file(filename = os.path.join(path,'StreamStats_Polygons.shp')) #Export the geodataframe as a shapefile

C:\Users\sputnam\AppData\Local\Continuum\anaconda3\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


##### The catchment polygons as a geojson:

In [13]:
with open(os.path.join(path,'StreamStats_Polygons.geojson'), 'w') as f:
     dump(gdf, f)  

# END